# Setting up structures with pyiron

This notebook shows how to set-up structures for a calculation using the modules available in pyiron

In [1]:
from pyiron_atomistics.structure.atoms import Atoms
import numpy as np
from pyiron.project import Project

In [2]:
pr = Project("structure_setup")

## Set-up of an oxygen dimer in a cubic cell

The most basic way to set-up a simulation cell is through the explicit input of the cell parameters and positions

In [3]:
cell = 10.0 * np.eye(3) # Specifying the cell dimensions
positions = [[0.25, 0.25, 0.25], [0.75, 0.75, 0.75]]
elements = ['O', 'O']

# Now use the Atoms class to create the instance. Specity if the coordinates are absolute or relative as well 
# as the periodic boundary conditions
O_dimer = Atoms(elements=elements, scaled_positions=positions, cell=cell)

print("cell volume:", O_dimer.get_volume())

cell volume: 1000.0


In [4]:
# Visualization through nglview is possible using the following command
view = O_dimer.plot3d()

## Compatability with ase

The PyIron Atoms class is compatible with that of the popular Atomistic Simulation Environment (ASE) module. This means that one 
can additionally use ASE to set-up and manipulate structures in combination with the functionalities of available in pyiron

In [5]:
# Setting up a diamond structure

Si_basis = pr.create_ase_bulk("Si")
view = Si_basis.plot3d()
view

NGLWidget()

In [6]:
len(Si_basis)

2

In [7]:
# Further manipulations like translational repetations, scaling etc can also be performed

Si_basis.set_repeat([4, 4, 4])
view = Si_basis.plot3d()
view.add_ball_and_stick()
view.remove_spacefill()
view

NGLWidget()

In [8]:
len(Si_basis)

128

## Setting up rock salt type structures (NaCl, MgO etc)

Here we create a rock-salt structure by creating two 'fcc' sublattices and shifting one of them by half a lattice parameter

In [9]:
a = 4.2

Mg_basis = pr.create_ase_bulk(name="Mg", crystalstructure="fcc", a=a, cubic=True)
O_basis = pr.create_ase_bulk(name="O", crystalstructure="fcc", a=a, cubic=True)

O_basis.scaled_positions += [0.5, 0, 0.]
MgO_basis = O_basis + Mg_basis
MgO_basis.set_repeat([2, 2, 2])
MgO_basis.center_coordinates_in_unit_cell()
print(MgO_basis.get_spacegroup())
view = MgO_basis.plot3d()
view

{'Number': 225, 'InternationalTableSymbol': 'Fm-3m'}


NGLWidget()

## Set-up of a Wurzite AlN structure

In [10]:
# Set-up of a Wurzite AlN structure
a = 3.11
c = np.sqrt(8. / 3.) * a
# Typical coordination of a particular species A
wyck_pos = np.array([[1./ 3., 2. / 3., 0.], [2. / 3., 1. / 3., 1. / 2.]])
# Shift of species B wrt A
u = [0., 0., 3. / 8.]
# Using a hexagonal lattice
hex_lattice = pr.create_ase_bulk('Mg', crystalstructure="hcp", a=a, c=c)
Al_basis = Atoms(elements=['Al', 'Al'], scaled_positions=wyck_pos, cell=hex_lattice.cell)
N_basis = Atoms(elements=['N', 'N'], scaled_positions=wyck_pos, cell=hex_lattice.cell)
# Shifting N basis
N_basis.scaled_positions += u
AlN_basis = Al_basis + N_basis
AlN_basis.set_repeat([2, 2, 2])
view = AlN_basis.plot3d()
view.add_ball_and_stick()
view

NGLWidget()

In [11]:
# Verify that the generated structure is indeed a wurzite structure by getting it's symmetry number
AlN_basis.get_spacegroup()

{'InternationalTableSymbol': 'P6_3mc', 'Number': 186}

## Creating defects and impurities

First let us create an O vacancy in MgO


In [12]:
MgO_basis_vac = MgO_basis.copy()
print(MgO_basis_vac.get_chemical_formula())
O_indices = MgO_basis_vac.select_index("O") 
vac_ind = np.random.choice(O_indices)
del MgO_basis_vac[vac_ind]
print(MgO_basis_vac.get_chemical_formula())

Mg32O32
Mg32O31


Then let us introduce a H interstitial at this site

In [13]:
MgO_basis_int = MgO_basis.copy()
MgO_basis_int[vac_ind] = "H"
print(MgO_basis_int.get_chemical_formula())

HMg32O31
